In [1]:
import pandas as pd
import numpy as np
import folium
import requests

In [2]:
data = pd.read_csv('Frederick_County_Real_Property_Assessments__Hidden_Property_Owner_Names.csv')
data = data[['MDP Longitude (MDP Field: DIGXCORD converted to WGS84)','MDP Latitude (MDP Field: DIGYCORD converted to WGS84)','BASE CYCLE DATA: Land Value (SDAT Field #154)']]

/tmp/ipykernel_335/2295617219.py:1: DtypeWarning: Columns (29,31,41,42,43,48,63,64,65,66,128,166,172,211) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Frederick_County_Real_Property_Assessments__Hidden_Property_Owner_Names.csv')


In [3]:
data = data.rename(columns = {'MDP Longitude (MDP Field: DIGXCORD converted to WGS84)': 'Longitude','MDP Latitude (MDP Field: DIGYCORD converted to WGS84)': 'Latitude','BASE CYCLE DATA: Land Value (SDAT Field #154)':'Land Value'}) 

Now I will check if there is any missing data in the table.

In [4]:
for col in data.columns:
    print(col +': ' + str(len(data[data[col].isna()==True])))

Longitude: 3836
Latitude: 3836
Land Value: 0


There is missing data but since I only plan to sample some of the points in order to not have the map over filled, I don't need all the points an can just take the sample from the ones that have a longitude and latitude. I need to take a sample for two reasons. One so the map is not to filled with points and you can still see different individual points fairly easy. Two because when trying top plot high number of points it kept crashing so I have chosen to take a sample of 1000 point. Assuming this is enough to get a good represetation of the properties in Frederick County, but a small enough amount to work. Since properties near each other normally sell for a similar value you don't need to look at every property in the neighborhood so looking at a sample will tell us alot about the surrounding properties as well.

In [5]:
data = data[data['Longitude'].isna() == False]
data = data.sample(1000)

We see above that the longitude and latitude columns are missing some values. About .5% of data is missing for those columns, but all have the land value. Therefore I will use that column in my imputation method. I will fill in the missing data by finding the closest two houses to the value of the on missing data and the average the longitude and latitude of those two. I chose to do it this way assuming that houses near each other will be worth around the same value, but I also didn't want to assign a longitude and lattitude exactly the same as a house with all the data filled in because then on the map they would over lap. That is why I did average of closest two so the longitude and latitude will be slighly different.

In [6]:
for col in data.columns:
    print(col +': ' + str(len(data[data[col].isna()==True])))

Longitude: 0
Latitude: 0
Land Value: 0


Now there is no longer any missing values in the data so we can get to making the map.

In [7]:
print(np.min(data['Land Value']))
print(np.max(data['Land Value']))

0.0
12482700.0


In [8]:
map_osm = folium.Map(location = [39.38, -77.47], zoom_start = 12)

In [9]:
for index, value in data.iterrows():
    if value["Land Value"] < 125000:
        folium.Marker(location=[value["Latitude"], value["Longitude"]],
                    icon=folium.Icon(color='red')).add_to(map_osm)
    elif value["Land Value"] >= 125000 and value["Land Value"]< 250000:
        folium.Marker(location=[value["Latitude"], value["Longitude"]],
            icon=folium.Icon(color='lightgreen')).add_to(map_osm)
    elif value["Land Value"] >= 250000 and value["Land Value"]< 500000:
        folium.Marker(location=[value["Latitude"], value["Longitude"]],
            icon=folium.Icon(color='blue')).add_to(map_osm)
    elif value["Land Value"] >= 500000 and value["Land Value"]< 1000000:
        folium.Marker(location=[value["Latitude"], value["Longitude"]],
            icon=folium.Icon(color='green')).add_to(map_osm)
    elif value["Land Value"] >= 1000000 and value["Land Value"]< 2000000:
        folium.Marker(location=[value["Latitude"], value["Longitude"]],
            icon=folium.Icon(color='orange')).add_to(map_osm)
    elif value["Land Value"] >= 2000000 and value["Land Value"]< 3000000:
        folium.Marker(location=[value["Latitude"], value["Longitude"]],
            icon=folium.Icon(color='purple')).add_to(map_osm)
    elif value["Land Value"] >= 3000000:
        folium.Marker(location=[value["Latitude"], value["Longitude"]],
            icon=folium.Icon(color='lightblue')).add_to(map_osm)
        
map_osm 